In [71]:
import pandas as pd
import numpy as np

In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [73]:
df = pd.read_csv("DBPEDIA_train.csv")

In [74]:
vectorizer = TfidfVectorizer(stop_words='english',
                             strip_accents='unicode',
                             norm='l2',
                             )

In [75]:
X_train = []
for i in range(len(df)):
  string = df['text'][i]            
  string = vectorizer.build_preprocessor()(string.lower()) 
  string = vectorizer.build_tokenizer()(string.lower())
  X_train.append(' '.join(string))

In [76]:
df_test = pd.read_csv("DBPEDIA_test.csv")

In [77]:
X_test = []
for i in range(len(df_test)):
  string = df_test['text'][i]
  string = vectorizer.build_preprocessor()(string.lower())
  string=vectorizer.build_tokenizer()(string.lower())
  X_test.append(' '.join(string))

In [78]:
X_train = vectorizer.fit_transform(X_train)

In [79]:
X_test = vectorizer.transform(X_test)

In [80]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
df['l1'] = LE.fit_transform(df['l1'])

In [81]:
df_test['l1'] = LE.transform(df_test['l1'])

In [82]:
y_train = df['l1']
y_test = df_test['l1']

In [83]:
X_train = X_train.todense()
y_train=y_train.todense()

MemoryError: Unable to allocate 846. GiB for an array with shape (240942, 471243) and data type float64

In [87]:
X_train_np = np.array(X_train)
y_train_np = np.array(y_train)

In [90]:
pip install tf-nightly

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ------------------------------------ 177.7/177.7 MB 451.5 kB/s eta 0:00:00
     ---------------------------------------- 6.0/6.0 MB 518.8 kB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 363.3 kB/s eta 0:00:00
     ------------------------------------ 439.4/439.4 kB 232.8 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\khair\\anaconda3\\Lib\\site-packages\\tensorflow\\compiler\\tf2tensorrt\\_pywrap_py_utils.pyd'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [91]:
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import tensorflow as tf

In [92]:
model = Sequential()
model.add(Embedding(471243,256 ,input_length = 471243))
model.add(LSTM(units=128, return_sequences = True))
model.add(Dense(64, activation="relu"))
model.add(LSTM(units=6, return_sequences=True))
model.add(LSTM(units=1, return_sequences=True, name='output'))
model.compile(loss=tf.keras.losses.KLDivergence(),optimizer='sgd', metrics = ['accuracy'])

In [93]:
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, 471243, 256)       120638208 
                                                                 
 lstm_16 (LSTM)              (None, 471243, 128)       197120    
                                                                 
 dense_3 (Dense)             (None, 471243, 64)        8256      
                                                                 
 lstm_17 (LSTM)              (None, 471243, 6)         1704      
                                                                 
 output (LSTM)               (None, 471243, 1)         32        
                                                                 
Total params: 120,845,320
Trainable params: 120,845,320
Non-trainable params: 0
_________________________________________________________________


In [64]:
model.fit_generator(generator=batch_generator(X_train, y_train, 32),
                    epochs=10)

C:\Users\khair\AppData\Local\Temp\ipykernel_12824\1238351121.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=batch_generator(X_train, y_train, 32),


KeyError: 'key of type tuple not found and not a MultiIndex'

In [59]:
def batch_generator(X, y, batch_size):
    n_batches_for_epoch = X.shape[0]//batch_size
    for i in range(n_batches_for_epoch):
        index_batch = range(X.shape[0])[batch_size*i:batch_size*(i+1)]       
        X_batch = X[index_batch,:].todense()
        y_batch = y[index_batch,:]
        yield(np.array(X_batch),y_batch)

In [95]:
model.fit(X_train,y_train)

InvalidArgumentError: Graph execution error:

TypeError: 'SparseTensor' object is not subscriptable
Traceback (most recent call last):

  File "C:\Users\khair\anaconda3\lib\site-packages\tensorflow\python\ops\script_ops.py", line 269, in __call__
    return func(device, token, args)

  File "C:\Users\khair\anaconda3\lib\site-packages\tensorflow\python\ops\script_ops.py", line 147, in __call__
    outputs = self._call(device, args)

  File "C:\Users\khair\anaconda3\lib\site-packages\tensorflow\python\ops\script_ops.py", line 154, in _call
    ret = self._func(*args)

  File "C:\Users\khair\anaconda3\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "C:\Users\khair\anaconda3\lib\site-packages\keras\engine\data_adapter.py", line 513, in py_method
    flat_out = tf.py_function(py_method, [indices], flat_dtypes)

  File "C:\Users\khair\anaconda3\lib\site-packages\keras\engine\data_adapter.py", line 513, in <listcomp>
  